In [1]:
print("Installing necessary libraries...")

# Transformers: Lets you easily use powerful pre-trained AI models for tasks like text generation, translation, or classification.
# Accelerate: Helps you run and train models efficiently across CPUs, GPUs.
# BitsAndBytes: Reduces the size of large models using quantization so they can fit and run on limited hardware like free GPUs.
# Torch (PyTorch): A deep learning library used to build, train, and run neural networks using tensors and GPU acceleration.

# Added 'datasets' to the list
!pip install -q transformers accelerate bitsandbytes torch datasets gradio

print("Libraries installed successfully!")

Installing necessary libraries...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.0 MB/s eta 0:00:00:00

In [2]:
# Hugging Face Login

import os
from huggingface_hub import login, notebook_login

print("Attempting Hugging Face login...")

notebook_login()
print("Login successful (or token already present)!")

Attempting Hugging Face login...


Login successful (or token already present)!


In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset  # Import the dataset loading function
import gradio as gr
from IPython.display import display, Markdown
import random  # To pick random news items

# Check for GPU availability
if torch.cuda.is_available():
    print(f"GPU detected: {torch.cuda.get_device_name(0)}")
    torch.set_default_device("cuda")
    print("PyTorch default device set to CUDA (GPU).")
else:
    print("WARNING: No GPU detected. Performance will be very slow.")
    print("Go to Runtime > Change runtime type and select GPU.")


2025-12-12 07:33:27.704450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765524807.932603      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765524808.002338      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

GPU detected: Tesla T4
PyTorch default device set to CUDA (GPU).


In [4]:
# Helper function for markdown display
def print_markdown(text):
    """Displays text as Markdown."""
    display(Markdown(text))

In [5]:
dataset_id = "mohit9999/all_news_finance_sm_1h2023_custom"

print(f"Loading dataset: {dataset_id}...")

# Load the dataset (will download if not cached)
# We might only need the 'train' split, specify split = 'train' if needed
# The datatype of news_dataset is datasets.Dataset (from the datasets library by Hugging Face).
news_dataset = load_dataset(dataset_id, split = "train")
print("Dataset loaded successfully!")


Loading dataset: mohit9999/all_news_finance_sm_1h2023_custom...


README.md:   0%|          | 0.00/349 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5062 [00:00<?, ? examples/s]

Dataset loaded successfully!


In [6]:
# Let's Inspect the dataset
news_dataset

Dataset({
    features: ['instruction', 'output', 'text'],
    num_rows: 5062
})

In [7]:
# Let's display the features (columns and their types)
print("\n Dataset Features")
print(news_dataset.features)


 Dataset Features
{'instruction': Value('string'), 'output': Value('string'), 'text': Value('string')}


In [8]:
# Let's view the datasets as a Pandas DataFrame
news_dataset.to_pandas()

,instruction,output,text
0,"Dow drops 400 points, turns negative for the y...",Regional banks led the broader market lower as...,Below is an instruction that describes a task....
1,Bitcoin drops with stocks as analyst warns of ...,Bitcoin dips as the U.S. banking crisis engulf...,Below is an instruction that describes a task....
2,Bitcoin Price Analysis: 29370 Tested After Su...,Bitcoin (BTC/USD) sought to add to recent gain...,Below is an instruction that describes a task....
3,"Bitcoin Is 75% To Halving, Here's How Past Cyc...",The current Bitcoin cycle is now 75% on the wa...,Below is an instruction that describes a task....
4,"Iron Mountain FFO of $0.71 beats by $0.03, rev...",Iron Mountain press release (IRM): Q1 FFO of...,Below is an instruction that describes a task....
...,...,...,...
5057,"Ethereum Merge can trigger high volatility, Bi...",BitMEX CEO Alexander Höptner said that their t...,Below is an instruction that describes a task....
5058,EXPLAINER: Is COVID-19 winding down? Scientist...,Is the coronavirus on its way out? You might t...,Below is an instruction that describes a task....
5059,How Bitcoin Educates The World About Finance,Bitcoin brings financial education to the fore...,Below is an instruction that describes a task....
5060,Crypto Hedge Fund Three Arrows Capital Wallet ...,The removal of liquidity was the first activit...,Below is an instruction that describes a task....


In [13]:
# Let's prepare the data for the LLM
# We'll combine instruction and output for the input text
def combine_news_text(example):

    # Handle potential None values gracefully
    instruction = example.get("instruction", "") or ""
    output = example.get("output", "") or ""

    # Add a separator for clarity
    return {"Full_text": f"Instruction: {instruction}\nOutput: {output}"}


In [14]:
# Let's apply the function to combine the 'title' and 'description' into 'full_text'
# This uses map, which is efficient for datasets

news_dataset = news_dataset.map(combine_news_text)

print("\n--- Sample Data with 'full_text' ---")
print(news_dataset[0])

Map:   0%|          | 0/5062 [00:00<?, ? examples/s]


--- Sample Data with 'full_text' ---
{'instruction': 'Dow drops 400 points, turns negative for the year as bank fears grow: Live updates', 'output': 'Regional banks led the broader market lower as contagion fears resurfaced.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction: Dow drops 400 points, turns negative for the year as bank fears grow: Live updates\n\n### Response: Regional banks led the broader market lower as contagion fears resurfaced.', 'Full_text': 'Instruction: Dow drops 400 points, turns negative for the year as bank fears grow: Live updates\nOutput: Regional banks led the broader market lower as contagion fears resurfaced.'}


In [15]:
# Let's display the full_text of the first sample
print(news_dataset[2]["Full_text"])

Instruction: Bitcoin Price Analysis:  29370 Tested After Surge - 5 May...
Output: Bitcoin (BTC/USD) sought to add to recent gains early in the Asian session as the pair extended recent gains to the 29383.50 area, representing a test of an upside p...


In [16]:
# Let's choose our smaller DeepSeek model, which is suitable for Google Colab and reasoning tasks.
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
print(f"Loading model: {model_id}")
print("This may take a few minutes...")


# AutoModelForCausalLM.from_pretrained(...): Loads a pre-trained language model for tasks like text generation (e.g., ChatGPT-like behavior).
# model_id: The name or path of the model you want to load from Hugging Face (like "meta-llama/Llama-3-8b").
# torch_dtype="auto": Automatically chooses the best data type (like float16 or float32) based on your hardware for efficiency.
# load_in_4bit=True: Loads the model in 4-bit precision to save memory and run on limited hardware like free Colab GPUs.
# device_map="auto": Automatically puts the model on the right device (GPU if available, otherwise CPU).
# trust_remote_code=True: Use only with trusted models to avoid security risks.

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype = "auto",
                                             load_in_4bit = True,
                                             device_map = "auto",
                                             trust_remote_code = True)

model.eval()
print("Model loaded successfully in 4-bit!")

Loading model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
This may take a few minutes...


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Model loaded successfully in 4-bit!


In [17]:
# Let's load the Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code = True)
print("Tokenizer loaded successfully.")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Tokenizer loaded successfully.


In [18]:
# Let's create the Text Generation Pipeline
llm_pipeline = pipeline("text-generation", model = model, tokenizer = tokenizer, torch_dtype = "auto", device_map = "auto")
print("Text generation pipeline created successfully.")

Device set to use cuda:0


Text generation pipeline created successfully.


In [19]:
# Let's build the full formatted prompt:
# <|im_start|>user: Marks the beginning of the user's message.
# {test_question}: Inserts your question.
# <|im_end|>: Marks the end of the user’s message.
# <|im_start|>assistant: Signals that the assistant (AI) is about to reply.
# <think>: Encourages the AI to generate internal reasoning or thoughts before answering.

test_question = "Explain what electric cars are in simple terms. Keep the thinking short."
test_prompt = f"<|im_start|>user\n{test_question}<|im_end|>\n<|im_start|>assistant\n<think>\n"
print(f"Test prompt:\n{test_prompt}")

Test prompt:
<|im_start|>user
Explain what electric cars are in simple terms. Keep the thinking short.<|im_end|>
<|im_start|>assistant
<think>



In [20]:
# Let's test this beast of a model!
# Generate text using the pipeline
# max_new_tokens: Maximum number of tokens to generate
# do_sample: Enable sampling for more diverse outputs (vs greedy decoding). Enables sampling, meaning the model won’t always pick the highest probability next word — instead, it adds some randomness to make responses more creative and diverse.
# temperature: Controls randomness (0.7 is balanced between creative and focused)
# top_p: Nucleus sampling parameter (0.9 keeps responses on topic while allowing variety). Where the model chooses from the top 90% of likely next words (not just the top one). Helps keep output fluent but varied.

print("Testing model with a simple instruction")

outputs = llm_pipeline(test_prompt,
                       max_new_tokens = 4000,
                       do_sample = True,
                       temperature = 0.7,
                       top_p = 0.9)

outputs

Testing model with a simple instruction


[{'generated_text': "<|im_start|>user\nExplain what electric cars are in simple terms. Keep the thinking short.<|im_end|>\n<|im_start|>assistant\n<think>\nOkay, so I'm trying to figure out what electric cars are in simple terms. I've heard the term before, but I'm not exactly sure what it means. Let me start by thinking about what I know. I know that cars have different types, like SUVs, trucks, and maybe even some that are hybrid. But I don't know much about electric cars.\n\nI remember hearing that some cars are powered by batteries. Maybe that's what electric cars are? So, if a car uses a battery to power it, then maybe it's electric. But wait, what makes a car electric? I think it's because the battery can provide power to the car's electrical system. So, if a car's battery is strong enough, it can power the car's lights, tires, and other electrical components.\n\nI also think that electric cars might have some features like automatic driving, which means the car can drive itself w

In [22]:
# Function to extract and format the reason and output
def format_model_output(output):

    # Extract the content within <think> tags as Reason
    reason_start = output.find("<think>") + len("<think>")
    reason_end = output.find("</think>")
    reason = output[reason_start:reason_end].strip()

    # Extract the content after </think> as Output
    output_start = reason_end + len("</think>")
    model_output_content = output[output_start:].strip()

    # Format the result
    reason = f"**Reason**:\n{reason}\n"
    output = f"**Output**:\n{model_output_content}"
    return reason, output

In [23]:
# Extract and print the generated text (only the response part)
full_output = outputs[0]["generated_text"]
reason, output = format_model_output(full_output)
print_markdown(reason + output)

**Reason**:
Okay, so I'm trying to figure out what electric cars are in simple terms. I've heard the term before, but I'm not exactly sure what it means. Let me start by thinking about what I know. I know that cars have different types, like SUVs, trucks, and maybe even some that are hybrid. But I don't know much about electric cars.

I remember hearing that some cars are powered by batteries. Maybe that's what electric cars are? So, if a car uses a battery to power it, then maybe it's electric. But wait, what makes a car electric? I think it's because the battery can provide power to the car's electrical system. So, if a car's battery is strong enough, it can power the car's lights, tires, and other electrical components.

I also think that electric cars might have some features like automatic driving, which means the car can drive itself without human control. That sounds familiar. Maybe that's how it works. Also, electric cars might be more fuel-efficient because they don't rely on combustion engines. That makes sense because combustion engines use fuel and oxygen to produce energy, which can be less efficient.

But I'm not sure if all cars are electric. There are hybrid cars too, which have a mix of an internal combustion engine and a battery. So, electric cars are a subset of cars. I should probably explain that in the answer.

I should also mention the benefits, like reduced emissions and lower fuel costs, which are pretty appealing. But I don't want to make it too technical. Keep it simple, as the user requested.

So, putting it all together, electric cars are cars that use a battery to power them. They might have automatic driving, be fuel-efficient, and could come in various shapes and sizes like SUVs, trucks, or hybrids. They use a battery, which is strong enough to power the car's electrical system. They might have features like automatic driving, which allows the car to drive itself without human control. Electric cars are different from other car types like hybrids or combustion engines, and they're popular because they're more fuel-efficient and can reduce emissions.

I think that covers it. I'll structure the answer to explain what electric cars are, their features, and their benefits in simple terms. I'll keep it short and straightforward as the user requested.
**Output**:
Electric cars are cars that use a battery to power them. They might have features like automatic driving, which allows the car to drive itself without human control. Electric cars are different from other car types like hybrids or combustion engines and are popular because they're more fuel-efficient and can reduce emissions.

In [24]:
import re  # Import regular expressions for parsing

def analyze_news_sentiment(news_text, llm_pipeline):
    """
    Analyzes news sentiment using the LLM, providing reasoning and classification.

    Args:
        news_text (str): The combined title and description of the news.
        llm_pipeline (transformers.pipeline): The initialized text-generation pipeline.

    Returns:
        str: A string containing the reasoning and classification.
    """

    # Define the Prompt Template with specific instructions for the model
    test_question = f"""You are a concise News Analyst.
    Analyze the provided news text. Output ONLY the three requested items below, each on a new line, prefixed with the specified label.

    1.  REASONING: Brief financial reasoning (1-2 points max) for the sentiment.
    2.  SENTIMENT: Choose ONE: Positive, Negative, Neutral.
    3.  TAG: A concise topic tag (1-3 words).

    Do NOT add any other text, greetings, or explanations.

    News Text:
    {news_text}"""

    # Format the prompt according to DeepSeek's expected input format with thinking tags
    test_prompt = f"<|im_start|>user\n{test_question}<|im_end|>\n<|im_start|>assistant\n<think>\n"

    # Print the prompt for debugging purposes to verify what's being sent to the model
    print(f"Test prompt:\n{test_prompt}")

    # Run the model inference with specific generation parameters
    outputs = llm_pipeline(
        test_prompt,
        max_new_tokens = 4000,
        do_sample = True,
        temperature = 0.7,
        top_p = 0.9)

    # Extract the full generated text and parse it to separate reasoning from classification
    # The format_model_output function likely separates the thinking process from the final answer
    full_output = outputs[0]["generated_text"]
    reason, output = format_model_output(full_output)

    # Return both the reasoning process and the final sentiment classification
    return reason, output


In [25]:

# Print a separator line for clarity in the output
print("\n" + "=" * 30 + " TESTING ANALYSIS " + "=" * 30)


# Select a few random indices from the news dataset to test the analysis function
random_indices = random.sample(range(len(news_dataset)), 3)

# Iterate over each randomly selected index
for index in random_indices:
    # Retrieve the full text of the news item at the current index
    sample_news = news_dataset[index]["Full_text"]

    # Analyze the sentiment of the sample news using the sentiment analysis function
    reason, output = analyze_news_sentiment(sample_news, llm_pipeline)

    # Print the analysis result header for the current index
    print(f"\n--- Analysis Result for Index {index} ---")

    # Display the reasoning and output in a formatted markdown style
    print_markdown(f"{reason}\n\n{output}")

    # Print a separator line for better readability between results
    print("-" * 60)




============================== TESTING ANALYSIS ==============================
Test prompt:
<|im_start|>user
You are a concise News Analyst.
    Analyze the provided news text. Output ONLY the three requested items below, each on a new line, prefixed with the specified label.

    1.  REASONING: Brief financial reasoning (1-2 points max) for the sentiment.
    2.  SENTIMENT: Choose ONE: Positive, Negative, Neutral.
    3.  TAG: A concise topic tag (1-3 words).

    Do NOT add any other text, greetings, or explanations.

    News Text:
    Instruction: WSJ News Exclusive | U.S. Considers Release of Intelligence on China’s Potential Arms Transfer to Russia
Output: The Biden administration is considering releasing intelligence it believes shows that China is weighing whether to supply weapons to support Russia’s war in Ukraine, U.S. officials said.<|im_end|>
<|im_start|>assistant
<think>


--- Analysis Result for Index 1654 ---


**Reason**:
Okay, I need to analyze this news text. The main points are about the U.S. considering releasing intelligence on China's potential arms transfer to Russia. The user wants me to extract three items: Reasoning, Sentiment, and Tag. 

First, the Reasoning. The text mentions releasing intelligence showing China's decision to transfer weapons to Russia's war. This shows a potential strategic move by China, which could influence U.S. relations. The reasoning is probably pointing out the strategic significance of such an action.

Next, Sentiment. The text indicates uncertainty and potential actions, so it's neutral. The reasoning supports a neutral stance as the situation isn't extremely positive or negative.

For the Tag, I need a concise topic. "Strategic Arms Transfer" seems fitting as it captures the essence of the news.

Putting it all together, I'll structure each point clearly with the reasons and sentiment.


**Output**:
1. REASONING: The text indicates that the U.S. is considering releasing intelligence suggesting China's potential decision to transfer weapons to Russia's war in Ukraine. This reflects a strategic consideration of arms transfer dynamics between China and Russia. The reasoning highlights the importance of China's potential move in shaping U.S. relations with Russia.

2. SENTIMENT: Neutral. The text suggests that the situation is neutral, with no strong positive or negative implications for the narrative.

3. TAG: Strategic Arms Transfer. This tag captures the essence of the news, focusing on China's potential arms transfer to Russia.

------------------------------------------------------------
Test prompt:
<|im_start|>user
You are a concise News Analyst.
    Analyze the provided news text. Output ONLY the three requested items below, each on a new line, prefixed with the specified label.

    1.  REASONING: Brief financial reasoning (1-2 points max) for the sentiment.
    2.  SENTIMENT: Choose ONE: Positive, Negative, Neutral.
    3.  TAG: A concise topic tag (1-3 words).

    Do NOT add any other text, greetings, or explanations.

    News Text:
    Instruction: The Largest Ethereum Whale Just Spent $7 Million Buying Shiba Inu | Bitcoinist.com
Output: Ethereum whales have been known to go on large Shiba Inu buying sprees in the past. These whales have made bank off of the meme coin following its meteoric<|im_end|>
<|im_start|>assistant
<think>


--- Analysis Result for Index 4982 ---


**Reason**:
Okay, so I need to analyze this news text about the whale buying Shiba Inu. The user wants me to output only three items: Reasoning, Sentiment, and TAG. Let me break it down step by step.

First, the Reasoning part. I should explain why I think the sentiment is negative. The text mentions that whales have bought Shiba Inu in the past, which is a big meme. I remember that Shiba Inu was known for its high price and not being stable, which can make investors nervous. The whale's purchase adds to this uncertainty, so the sentiment should be negative.

Next, the Sentiment. I need to choose between Positive, Negative, or Neutral. Since the whale's purchase is leading to concern about the meme's stability and market sentiment, it's clear the sentiment is Negative.

Lastly, the TAG. I should provide a concise topic tag. The main points are the whale's purchase and the meme's impact, so something like "Meme Impact" seems appropriate.

I think that covers everything. I should make sure each part is clearly labeled and concise.


**Output**:
1. REASONING: The whale's purchase of Shiba Inu has created significant uncertainty and concern, leading to a negative sentiment.

2. SENTIMENT: Negative

3. TAG: Meme Impact

------------------------------------------------------------
Test prompt:
<|im_start|>user
You are a concise News Analyst.
    Analyze the provided news text. Output ONLY the three requested items below, each on a new line, prefixed with the specified label.

    1.  REASONING: Brief financial reasoning (1-2 points max) for the sentiment.
    2.  SENTIMENT: Choose ONE: Positive, Negative, Neutral.
    3.  TAG: A concise topic tag (1-3 words).

    Do NOT add any other text, greetings, or explanations.

    News Text:
    Instruction: Tesla stock slumps, investors lash out as Elon Musk focuses on Twitter
Output: Tesla's stock is down about 65 percent for the year and Musk is still selling millions of shares.<|im_end|>
<|im_start|>assistant
<think>


--- Analysis Result for Index 3618 ---


**Reason**:
Okay, so I need to analyze this news text about Tesla stock slumping. Let me read through it carefully. It says Tesla's stock is down about 65 percent for the year, and Musk is still selling millions of shares. The user wants me to output three items: the reasoning, sentiment, and tag. 

First, the reasoning. I should explain why the stock is down. Well, Tesla's stock has been affected by various factors. I remember that Tesla has a lot of market capitalization, so even a downturn can cause significant losses. I also think that the company has been dealing with financial pressures, maybe even going through some restructuring. The reason investors are selling shares is likely because they see the company's fundamentals as weak. Maybe the leadership at Tesla isn't as strong, or there are other issues affecting the company.

Next, the sentiment. The stock has gone down, which is a negative trend. I don't see any positive signs mentioned. So, I should choose Negative.

Lastly, the tag. I need a concise topic tag. The main points are the stock drop and the reasons behind it, so maybe "Stock Drop" would work.

I should make sure to only output each item on a new line, without any extra text. Also, keep it simple and clear. Let me put that together.


**Output**:
1. REASONING: Tesla's stock has declined significantly, reflecting a downward trend driven by financial pressures and leadership challenges at the company. Investors are selling shares due to perceived weak fundamentals.  
2. SENTIMENT: Negative.  
3. TAG: Stock Drop

------------------------------------------------------------
